1. Go to: https://colab.research.google.com/drive/1o-Ao3_oe0CKDDUJlDplGvhB70wcKaBMA
2. Click "Open in playground"
3. Click run (and then Run anyway)
4. Scroll down to the bottom and follow the program's instructions

In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from datetime import datetime

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import uuid
import os

class GoogleDriveImageDatabase:
  def __init__(self, DATABASE_GID):
    assert isinstance(DATABASE_GID, str)
    self.folders = {}
    folder_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(DATABASE_GID)}).GetList()
    for file in folder_list:
      if file['mimeType'] == "application/vnd.google-apps.folder":
        self.folders[file['title']] = file['id']
    print("{} folders loaded".format(len(self.folders.keys())))
  def upload(self, filename, character):
    assert isinstance(filename, str)
    assert isinstance(character, str)
    assert os.path.isfile(filename), "{} does not exist as a file".format(filename)
    assert self.checkFolder(character), "{} is not a valid character. Pick from list: \n{}".format(character, tuple(self.folders.keys()))
    file = drive.CreateFile({
        "title" : character + "_" + os.path.splitext(filename)[0],
        "mimeType" : "image/jpeg",
        "parents" : [{"id" : self.folders[character]}]
    })
    file.SetContentFile(filename)
    print("Attempting to upload {}".format(filename))
    file.Upload()
    os.remove(filename)
    print("{} uploaded and deleted".format(filename))

  def checkFolder(self, name):
    assert isinstance(name, str)
    return name in self.folders.keys()
    
def take_photos(character, picture_count, latency=1000.0):
  assert isinstance(character, str)
  assert isinstance(picture_count, int)
  assert isinstance(latency, float) or isinstance(latency, int)
  assert "." not in character, "File extension may not be specified. Will always be .jpg"
  assert picture_count >= 1, "Must take at least one picture"
  assert latency >= 1, "Latency cannot be negative or zero"
  js = Javascript('''
    async function takePhoto(picture_count, latency) {
      function logGenerator(parentDiv){
        var div = document.createElement("div")
        div.style.color = "red"
        function log(message){
          div.innerText = message
        }
        parentDiv.appendChild(div)
        return log
      }
      const quality = 1.0
      var red_snap_effect_length = 400;
      const div = document.createElement('div');
      div.width = "100%;"
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);
      logger = logGenerator(div);
      logger("Started initial script")

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      logger("Began stream")

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      logger("Awaiting capture")
      await new Promise((resolve) => capture.onclick = resolve);
      capture.disabled = true;
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      logger("Canvas prepared")
      var photos = [];
      if(latency > red_snap_effect_length){
        var actual_latency = latency - red_snap_effect_length;
      }else{
        var actual_latency = latency;
        red_snap_effect_length = 0;
      }
      logger("Starting pictures")
      for(var i = picture_count; i > 0; i--){
        // Wait for latency
        await new Promise(resolve => setTimeout(resolve, actual_latency))
        video.style.border = "solid red 2px"
        canvas.getContext('2d').drawImage(video, 0, 0);
        photos.push(canvas.toDataURL('image/jpeg', quality));
        picture_count--;
        logger(String(picture_count) + " pictures remain")
        await new Promise(resolve => setTimeout(resolve, red_snap_effect_length))
        if(red_snap_effect_length > 0){
          video.style.border = ""
        }

      }
      logger("Pictures complete")
      stream.getVideoTracks()[0].stop();
      logger("Video stopped")
      div.remove();
      logger("Script complete")
      return photos;
    }
    ''')
  display(js)
  data = eval_js('takePhoto({} ,{})'.format(picture_count, latency))
  print("Javascript completed")
  file_name_generator = lambda character, increment, date_obj: character + "_" +  date_obj.strftime("%m_%d_%Y_%H_%M_%S") + "_" +  str(increment) + ".jpg"
  i = 0
  date = datetime.now()
  files = []
  for file_data in data:
    binary = b64decode(file_data.split(',')[1])
    file_name = file_name_generator(character, i, date)
    with open(file_name, 'wb') as file:
      print("{} saved".format(file_name))
      file.write(binary)
      files.append(file_name)
    i += 1
  return files

from IPython.display import Image

auth.authenticate_user() # Google auth stuff, make sure to sign in with your ucsb account
gauth = GoogleAuth() # Google auth stuff
gauth.credentials = GoogleCredentials.get_application_default() # Google auth stuff
drive = GoogleDrive(gauth) # Google auth stuff
DATABASE_GID = "1RqiJwO6i1KJx54tRC3QJcHWjiooCLJNC" # GID of 'SigNN Character Database folder': https://drive.google.com/drive/folders/1RqiJwO6i1KJx54tRC3QJcHWjiooCLJNC?usp=sharing
database = GoogleDriveImageDatabase(DATABASE_GID) # Creates a database object which has method: .upload()

def getInput(message, validatorFunction, typing):
  value = None
  while True:
    value = input(message + "\n")
    try:
      value = typing(value)
    except:
      print("{} must be of type {}".format(value, typing))
      continue
    error = validatorFunction(value)
    if error == None:
      return value
    else:
      print("{} is an invalid. {}".format(value, error))

def validateCharacterExists(character):
  if database.checkFolder(character):
    return None
  return "Pick from a valid folder: {}".format(tuple(database.folders.keys()))

def validatePictureNumbers(picture_number):
  MAX_PICTURE_COUNT = 30
  if picture_number > MAX_PICTURE_COUNT:
    return "Cannot take more than {} pictures".format(MAX_PICTURE_COUNT)
  if picture_number < 1:
    return "Must take 1 picture or more"
  return None

def validatePictureDelay(picture_delay):
  MIN_PICTURE_DELAY = 100
  MAX_PICTURE_DELAY = 3000
  if picture_delay < MIN_PICTURE_DELAY:
    return "Delay cannot be any less than {} miliseconds. The pictures will be too similar.".format(MIN_PICTURE_DELAY)
  if picture_delay > MAX_PICTURE_DELAY:
    return "Delay cannot be any greater than {} miliseconds. There is no need to wait that long between pictures".format(MAX_PICTURE_DELAY)
  return None

while True:
  CHARACTER = getInput("Which character?", validateCharacterExists, str)
  NUMBER_OF_PICTURES = getInput("How many pictures?", validatePictureNumbers, int)
  DELAY_BETWEEN_PICTURES_MILISECONDS = getInput("What should the delay be between pictures (in miliseconds), (recomended = 500)", validatePictureDelay, float)
  while True:
    print("Camera will be open for {} seconds".format(NUMBER_OF_PICTURES * DELAY_BETWEEN_PICTURES_MILISECONDS / 1000))
    filenames = take_photos(CHARACTER, NUMBER_OF_PICTURES, DELAY_BETWEEN_PICTURES_MILISECONDS)
    for file in filenames:
      display(Image(file))
      database.upload(file, CHARACTER)
    
    cont = input("Continue taking '{}' pictures for '{}' with '{}' second delay? Type 'n' or 'no' to reconfigure. Any other input will result in continuing\n".format(NUMBER_OF_PICTURES, CHARACTER, DELAY_BETWEEN_PICTURES_MILISECONDS / 1000)).lower()
    if cont == "n" or cont == "no":
      break



  




In [0]:
filename